In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
fake = faker.Faker(locale='en')

In [3]:
microsoft_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'host': 'careers.microsoft.com',
    'referer': 'http://careers.microsoft.com/us/en'
}

In [4]:
SEARCH_URL = "http://careers.microsoft.com/us/en/search-results?s=1"
JOB_URL = "http://careers.microsoft.com/us/en/job/"

In [5]:
pattern = re.compile(r'(\{\"jobs\".*).\"aggregations\"', re.MULTILINE)

In [6]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

In [7]:
job_dicts = []
null_counter = 0
for shift in tqdm(range(0, 10_000, 50)):
    search_params = {
        'from': shift,
    }
    result = requests.get(SEARCH_URL, headers=microsoft_headers, params=search_params)
    soup = BeautifulSoup(result.text)

    found = re.search(pattern, soup.find('script').string).group(1) + "}"
    found = json.loads(found)

    if len(found['jobs']) == 0:
        null_counter += 1
        if null_counter == 3:
            break
    else:
        null_counter = 0

    for job_info in found['jobs']:

        try:
            job_dict = {
                'title': job_info['title'],
                'internal_id' : job_info['jobId'],
                'url': JOB_URL + job_info['jobId'],
                'description': clear_string(job_info['description']),
                'responsibilities': clear_string(job_info['jobResponsibilities']) if 'jobResponsibilities' in job_info.keys() else None,
                'qualifications': clear_string(job_info['jobQualifications']) if 'jobQualifications' in job_info.keys() else None,
                'company': job_info['companyName'],
                'publish_date': parser.parse(job_info['postedDate'])
            }
            job_dicts.append(job_dict)
        except:
            print(f'Cannot parse {JOB_URL + job_info["jobId"]}')

    time.sleep(0.2) # sleep for 200 ms

 77%|███████▋  | 154/200 [05:42<01:42,  2.22s/it]


In [8]:
snapshot = pd.DataFrame(filter(lambda x: x is not None, job_dicts))
print(len(snapshot))
snapshot.sample(5)

7553


,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
6584,Sr. Software Engineer,1008051,http://careers.microsoft.com/us/en/job/1008051,Azure Edge + Platform brings together Edge pla...,"Responsibilities: Architect, design and develo...",Required Qualifications: BS Degree in Computer...,Microsoft,2022-02-22 18:46:00
4064,Director - Technical Support Advisors (Modern ...,1344184,http://careers.microsoft.com/us/en/job/1344184,Microsoft is on a mission to empower every per...,You will be responsible for ensuring technical...,The following experience and skills are requir...,Microsoft,2022-05-23 00:00:00
7069,Principal Software Engineering Manager-(Azure ...,988196,http://careers.microsoft.com/us/en/job/988196,Engineering Manager&nbsp; &ndash; &nbsp; Devic...,This is a SWE manager role with lots of execut...,You must possess the following attributes: ...,Microsoft,2021-12-15 13:17:00
4348,Datacenter Technician I,1293103,http://careers.microsoft.com/us/en/job/1293103,"In alignment with our Microsoft values, we are...","As a successful Datacenter Technician, your su...","For this role, you are welcome to apply if you...",Microsoft,2022-05-16 21:22:00
5594,"Area Manager, Construction Safety",1270120,http://careers.microsoft.com/us/en/job/1270120,Microsoft&rsquo;s Cloud Operations &amp; Innov...,"Responsibilities: &nbsp; Develop, own and driv...",Minimum Qualifications: &nbsp; A University/Ba...,Microsoft,2022-04-13 18:36:00


In [9]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7553 entries, 0 to 7552
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             7553 non-null   object        
 1   internal_id       7553 non-null   object        
 2   url               7553 non-null   object        
 3   description       7553 non-null   object        
 4   responsibilities  7544 non-null   object        
 5   qualifications    7553 non-null   object        
 6   company           7553 non-null   object        
 7   publish_date      7553 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 472.2+ KB


In [10]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'02-07-2022'

In [11]:
snapshot.to_csv(f'../data/microsoft/{current_date}.csv')
# snapshot.to_csv(f'../data/microsoft/{current_date}.tsv', sep='\t')

#### Проверка на единичной вакансии

In [207]:
search_params = {
    'from': 0,
}

In [208]:
result = requests.get(SEARCH_URL, headers=microsoft_headers, params=search_params)
result

<Response [200]>

In [209]:
soup = BeautifulSoup(result.text)

In [210]:
soup.find('script').string

' /*<!--*/ var phApp = phApp || {"widgetApiEndpoint":"https://careers.microsoft.com/widgets","country":"us","deviceType":"desktop","locale":"en_us","absUrl":true,"refNum":"MICRUS","cdnUrl":"https://prodcmscdn.azureedge.net/careerconnectresources/p","baseUrl":"https://careers.microsoft.com/us/en/","baseDomain":"https://careers.microsoft.com","phenomTrackURL":"careers.microsoft.com/us/en/phenomtrack.min.js","pageName":"search-results","siteType":"external","rootDomain":"https://careers.microsoft.com","pageId":"page19"}; phApp.ddo = {"siteConfig":{"status":"success","errorCode":null,"errorMsg":null,"data":{"urlMap":{"home":"home","category":"c/:category","job":"job/:jobSeqNo/:title","jobcart":"jobcart","search-results":"search-results","glassdoor-reviews":"glassdoor-reviews"},"categoryUrlMap":{"MICRUS_Legal---Corporate-Affairs":"c/legal-corporate-affairs-jobs","MICRUS_Supply-Chain---Operations-Management":"c/supply-chain-operations-management-jobs","MICRUS_Field-Business-Leadership":"c/fi

здесь в phApp.ddo есть описания вакансий, но нужено парсить джаваскрипт... конкретнее нас инетерсует ключ eagerLoadRefineSearch внутри оson

In [212]:
# soup.find('script').string

In [213]:
found = re.search(pattern, soup.find('script').string).group(1) + "}"
found = json.loads(found)
len(found['jobs'])

50

In [214]:
[(i, j['title']) for i, j in enumerate(found['jobs'])]

[(0, 'Commercial Business Lead'),
 (1, 'Data Center Critical Environment Technician Manager'),
 (2, 'Support Engineer, Azure Synapse'),
 (3, 'Critical Environments Operations Manager (CEOM)'),
 (4, 'Support Engineer, Azure SQL Database (PaaS)'),
 (5, 'Data Center Inventory & Asset Technician (DIAT)'),
 (6, 'Account Director (FSI)'),
 (7, 'Senior Principal Customer Success Account Manager'),
 (8, 'Senior Software Engineer- SONiC'),
 (9, 'Supply Chain Analyst'),
 (10, 'M&A Group Compliance Manager'),
 (11, 'Principal Program Manager'),
 (12, 'Support Engineer ‐Azure Identity'),
 (13, 'Project Manager, Data Centre Integration'),
 (14, 'Sr. Manager, M&A and Venture Finance'),
 (15, 'Support Engineer - Azure Monitoring'),
 (16, 'Sourcing Engineer'),
 (17, 'Azure Customer Engineer (Fasttrack for Azure – Infra)  ANZ'),
 (18, 'GIV Strategy & Ops Sr. Program Manager'),
 (19, 'GIV Deal Operations and Economics Director'),
 (20, 'Senior Computer Architect – Architectural Analysis and Methodology'

In [232]:
parser.parse(found['jobs'][0]['postedDate'])

datetime.datetime(2022, 6, 24, 2, 52)

In [243]:
found['jobs'][0].keys()

dict_keys(['country', 'subCategory', 'city', 'companyName', 'description', 'industry', 'title', 'multi_location', 'type', 'orgFunction', 'experience', 'locale', 'multi_location_array', 'jobQualifications', 'jobSeqNo', 'postedDate', 'searchresults_display', 'descriptionTeaser', 'jobResponsibilities', 'dateCreated', 'state', 'jd_display', 'employmentType', 'parentRefNum', 'jobSummary', 'reqId', 'badge', 'jobId', 'isMultiLocation', 'jobVisibility', 'refNum', 'mostpopular', 'applyUrl', 'worksite', 'location', 'category', 'locationLatlong', 'requisitionRoleType'])

In [246]:
'state' in found['jobs'][0].keys()

True

In [236]:
print(clear_string(found['jobs'][0]['jobQualifications']))

The successful candidate is an experienced sales and marketing leader with deep SMB customer insights, a successful track record of business strategy, sales leadership and hands-on execution excellence. Specifically, this highly visible and strategic role requires a candidate, who possesses the following characteristics:Ability to set and drive customer strategy, thus translating strategy into concrete actions and marketing/sales motionsExcellent business judgment and results across a range of disciplines including multi-channel sales, marketing, finance, and strategyAbility to effectively drive cross-group collaboration, using project management, impact and influence, as well as communication skills.Strong analytical mindset, but creative in problem solving and entrepreneurial in approach. Effective at managing and thriving in ambiguous situations.Demonstration of continuous learning & growthAdditional requirements for this role include:10+ years&rsquo; experience in IT sales and mark